<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/CNN-LSTM-intra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.layers import Conv1D, Dense, MaxPooling1D, LSTM
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = pd.read_csv('/content/drive/MyDrive/intravehicle.csv')

In [4]:
y_data = data.pop('4')
x_data = data

In [5]:
train_x,test_x,train_y,test_y = train_test_split(x_data, y_data, test_size=0.2, random_state=0, shuffle=True) 

In [6]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True) 
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [7]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [8]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [9]:
model = Sequential()
model.add(Conv1D(activation="tanh", input_shape=(len(x_data.columns),1), kernel_size=2, filters=100))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100, activation="tanh", recurrent_activation="sigmoid"))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 3, 100)            300       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1, 100)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 81,104
Trainable params: 81,104
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_x, train_y, batch_size=32, epochs=6)

Epoch 1/6
292075/414237 [====================>.........] - ETA: 7:49 - loss: 0.1060 - accuracy: 0.9455

In [ ]:
pred = model.predict(test_x, verbose=1)
pred_y = np.argmax(pred, axis=2)

In [ ]:
confusion_matrix(test_y, pred_y)

In [ ]:
report = classification_report(test_y, pred_y)
print(report)